In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
torch.manual_seed(0)

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 50

In [3]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [4]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
    if t%5==4:
        torch.save(model.state_dict(), "/cluster/2023rreddy/ML-Course/Unit8/cloth_model.pt")
        model.load_state_dict(torch.load("/cluster/2023rreddy/ML-Course/Unit8/cloth_model.pt"))
print("Done!")

Epoch 1
-------------------------------
loss: 2.305311  [   64/60000]
loss: 2.298351  [ 6464/60000]
loss: 2.277986  [12864/60000]
loss: 2.275867  [19264/60000]
loss: 2.254384  [25664/60000]
loss: 2.230112  [32064/60000]
loss: 2.230498  [38464/60000]
loss: 2.198088  [44864/60000]
loss: 2.203432  [51264/60000]
loss: 2.172682  [57664/60000]
Test Error: 
 Accuracy: 49.5%, Avg loss: 2.162512 

Epoch 2
-------------------------------
loss: 2.174578  [   64/60000]
loss: 2.168033  [ 6464/60000]
loss: 2.111102  [12864/60000]
loss: 2.124136  [19264/60000]
loss: 2.076459  [25664/60000]
loss: 2.022723  [32064/60000]
loss: 2.042089  [38464/60000]
loss: 1.967449  [44864/60000]
loss: 1.975449  [51264/60000]
loss: 1.898246  [57664/60000]
Test Error: 
 Accuracy: 59.2%, Avg loss: 1.899494 

Epoch 3
-------------------------------
loss: 1.935572  [   64/60000]
loss: 1.908893  [ 6464/60000]
loss: 1.791451  [12864/60000]
loss: 1.824324  [19264/60000]
loss: 1.725938  [25664/60000]
loss: 1.674583  [32064/600

In [5]:
with torch.no_grad():
    for param in model.parameters():
        print(param.shape, param.numel())

torch.Size([512, 784]) 401408
torch.Size([512]) 512
torch.Size([512, 512]) 262144
torch.Size([512]) 512
torch.Size([10, 512]) 5120
torch.Size([10]) 10
